# Objective: Create a baseline model
# To-do: Predict the item sales of every item in November 2015

# Import

In [32]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

item_cat = pd.read_csv("/Users/leo/samurai/kaggle/pfs/data/item_categories.csv")
items = pd.read_csv("/Users/leo/samurai/kaggle/pfs/data/items.csv")
train = pd.read_csv("/Users/leo/samurai/kaggle/pfs/data/sales_train.csv")
shops = pd.read_csv("/Users/leo/samurai/kaggle/pfs/data/shops.csv")
test = pd.read_csv("/Users/leo/samurai/kaggle/pfs/data/test.csv")
sample_sub = pd.read_csv("/Users/leo/samurai/kaggle/pfs/data/sample_submission.csv")

# Check goal

In [34]:
sample_sub.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


# EDA

## item_cat

In [35]:
item_cat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [36]:
item_cat.shape

(84, 2)

In [37]:
item_cat.dtypes

item_category_name    object
item_category_id       int64
dtype: object

In [38]:
item_cat.isna().sum()

item_category_name    0
item_category_id      0
dtype: int64

In [39]:
item_cat.item_category_name.nunique()

84

In [40]:
item_cat.item_category_name.unique()[:5].tolist()

['PC - Гарнитуры/Наушники',
 'Аксессуары - PS2',
 'Аксессуары - PS3',
 'Аксессуары - PS4',
 'Аксессуары - PSP']

In [41]:
item_cat.item_category_name.value_counts()

Игровые консоли - PSP                  1
Игры - PS4                             1
Программы - MAC (Цифра)                1
Музыка - CD локального производства    1
Карты оплаты - Windows (Цифра)         1
                                      ..
Кино - Blu-Ray 3D                      1
Служебные                              1
Музыка - Подарочные издания            1
Кино - DVD                             1
Книги - Цифра                          1
Name: item_category_name, Length: 84, dtype: int64

In [42]:
item_cat["big_category"] = [
    x.split("-")[0].strip() for x in item_cat.item_category_name
]

In [43]:
item_cat.head()

,item_category_name,item_category_id,big_category
0,PC - Гарнитуры/Наушники,0,PC
1,Аксессуары - PS2,1,Аксессуары
2,Аксессуары - PS3,2,Аксессуары
3,Аксессуары - PS4,3,Аксессуары
4,Аксессуары - PSP,4,Аксессуары


In [44]:
item_cat.big_category.nunique()

20

In [49]:
item_cat.big_category.value_counts()

Книги                                13
Подарки                              12
Игры                                  8
Игровые консоли                       8
Аксессуары                            7
Программы                             6
Музыка                                6
Кино                                  5
Карты оплаты                          4
Игры PC                               4
Служебные                             2
Доставка товара                       1
Карты оплаты (Кино, Музыка, Игры)     1
Чистые носители (шпиль)               1
Элементы питания                      1
Игры Android                          1
Игры MAC                              1
Билеты (Цифра)                        1
PC                                    1
Чистые носители (штучные)             1
Name: big_category, dtype: int64

In [61]:
rus_eng = {
    "Книги": "books",
    "Подарки": "present",
    "Игры": "games",
    "Игровые консоли": "game consoles",
    "Аксессуары": "accesories",
    "Программы": "programs",
    "Музыка": "music",
    "Кино": "cinema",
    "Карты оплаты": "gift_cards",
    "Игры PC": "pc_games",
    "Служебные": "services",
    "Доставка товара": "delivery",
    "Карты оплаты (Кино, Музыка, Игры)": "payment_cards",
    "Чистые носители (шпиль)": "cd",
    "Элементы питания": "battery",
    "Игры Android": "android_games",
    "Игры MAC": "mac_games",
    "Билеты (Цифра)": "tickets",
    "PC": "pc",
    "Чистые носители (штучные)": "dvd",
}

In [64]:
item_cat.big_category.replace(to_replace=rus_eng, inplace=True)

In [70]:
item_cat.big_category.value_counts()

books            13
present          12
game consoles     8
games             8
accesories        7
music             6
programs          6
cinema            5
pc_games          4
gift_cards        4
services          2
battery           1
dvd               1
cd                1
pc                1
mac_games         1
android_games     1
delivery          1
tickets           1
payment_cards     1
Name: big_category, dtype: int64

In [71]:
item_cat

,item_category_name,item_category_id,big_category
0,PC - Гарнитуры/Наушники,0,pc
1,Аксессуары - PS2,1,accesories
2,Аксессуары - PS3,2,accesories
3,Аксессуары - PS4,3,accesories
4,Аксессуары - PSP,4,accesories
...,...,...,...
79,Служебные,79,services
80,Служебные - Билеты,80,services
81,Чистые носители (шпиль),81,cd
82,Чистые носители (штучные),82,dvd


## items

In [72]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [74]:
items.shape

(22170, 3)

In [76]:
items.nunique()

item_name           22170
item_id             22170
item_category_id       84
dtype: int64

## train

In [78]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [80]:
train.shape

(2935849, 6)

In [83]:
train[['date_block_num','shop_id','item_id','item_cnt_day']].groupby(['date_block_num','shop_id','item_id']).sum()

item_cnt_day
date_block_num shop_id item_id              
0              0       32                6.0
                       33                3.0
                       35                1.0
                       43                1.0
                       51                2.0
...                                      ...
33             59      22087             6.0
                       22088             2.0
                       22091             1.0
                       22100             1.0
                       22102             1.0

[1609124 rows x 1 columns]

## shops

## test

# Feature Engineering

# Validation

# Submission